<a href="https://colab.research.google.com/github/mhjalili84/Federated-Learning/blob/main/Model%20with%20db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Importing Required Libraries

In [ ]:
import sqlite3
import openai

#Step 2: Setting the API Key

In [ ]:
openai.api_key = "YOUR_OPENAI_API_KEY"

#Step 3: Database Initialization

In [ ]:
def init_db():
    conn = sqlite3.connect('chatbot.db')
    c = conn.cursor()

    # ایجاد جدول کاربران
    c.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    email TEXT NOT NULL UNIQUE,
                    age INTEGER,
                    gender TEXT
                )''')

    # ایجاد جدول گفتگوها
    c.execute('''CREATE TABLE IF NOT EXISTS conversations (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_email TEXT,
                    message TEXT,
                    is_bot INTEGER,  -- 1 for bot, 0 for user
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (user_email) REFERENCES users(email)
                )''')

    # ایجاد جدول تست‌ها
    c.execute('''CREATE TABLE IF NOT EXISTS test_results (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_email TEXT,
                    test_name TEXT,
                    score INTEGER,
                    result TEXT,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (user_email) REFERENCES users(email)
                )''')

    conn.commit()
    conn.close()

#Step 4: Adding a New User

In [ ]:
def add_user(name, email, age, gender):
    conn = sqlite3.connect('chatbot.db')
    c = conn.cursor()
    c.execute('INSERT INTO users (name, email, age, gender) VALUES (?, ?, ?, ?)',
              (name, email, age, gender))
    conn.commit()
    conn.close()

#Step 5: Saving Messages

In [ ]:
def save_message(user_email, message, is_bot):
    conn = sqlite3.connect('chatbot.db')
    c = conn.cursor()
    c.execute('INSERT INTO conversations (user_email, message, is_bot) VALUES (?, ?, ?)',
              (user_email, message, is_bot))
    conn.commit()
    conn.close()

#Step 6: Retrieving Chat History

In [ ]:
def get_conversation_history(user_email):
    conn = sqlite3.connect('chatbot.db')
    c = conn.cursor()
    c.execute('SELECT message, is_bot, timestamp FROM conversations WHERE user_email = ? ORDER BY timestamp ASC', (user_email,))
    rows = c.fetchall()
    for row in rows:
        role = "Bot" if row[1] == 1 else "User"
        print(f"{role} ({row[2]}): {row[0]}")
    conn.close()

#Step 7: Saving Test Results

In [ ]:
def save_test_result(user_email, test_name, score, result):
    conn = sqlite3.connect('chatbot.db')
    c = conn.cursor()
    c.execute('INSERT INTO test_results (user_email, test_name, score, result) VALUES (?, ?, ?, ?)',
              (user_email, test_name, score, result))
    conn.commit()
    conn.close()

#Step 8: Sentiment Analysis with OpenAI

In [ ]:
def analyze_sentiment(message):
    response = openai.Completion.create(
        engine="text-davinci-003",  # مدل مورد نظر
        prompt=f"Analyze the sentiment of the following text: {message}\nSentiment:",
        max_tokens=60
    )
    sentiment = response.choices[0].text.strip()
    return sentiment

#Step 9: Generating Bot Response

In [ ]:
def get_bot_response(user_message):
    response = openai.Completion.create(
        engine="text-davinci-003",  # مدل مورد نظر
        prompt=f"User says: {user_message}\nBot responds:",
        max_tokens=150
    )
    return response.choices[0].text.strip()

#Step 10: User Registration Input

In [ ]:
def register_user():
    print("ثبت‌نام کاربر")
    name = input("نام: ")
    email = input("ایمیل: ")
    age = int(input("سن: "))
    gender = input("جنسیت (مرد/زن/سایر): ")
    add_user(name, email, age, gender)
    return email

#Step 11: Chatting with the Bot

In [ ]:
def chat_with_bot(user_email):
    print("شروع گفتگو با بات (برای خروج، 'خروج' را تایپ کنید)")
    while True:
        user_msg = input("شما: ")
        if user_msg.lower() == "خروج":
            break
        save_message(user_email, user_msg, is_bot=0)

        # تحلیل احساسات
        sentiment = analyze_sentiment(user_msg)
        print(f"احساسات شناسایی‌شده: {sentiment}")

        # پاسخ‌دهی بات
        bot_response = get_bot_response(user_msg)
        print(f"بات: {bot_response}")
        save_message(user_email, bot_response, is_bot=1)

# Step 12: Psychological Test Example

In [ ]:
def run_test(user_email):
    print("تست اضطراب (پاسخ با عدد 0 تا 3)")
    score = 0
    questions = [
        "احساس نگرانی یا استرس دارید؟",
        "آیا دچار بی‌خوابی شده‌اید؟",
        "آیا تمرکزتان کاهش یافته؟"
    ]
    for q in questions:
        ans = int(input(q + " "))
        score += ans
    result = "کم" if score < 3 else "متوسط" if score < 6 else "زیاد"
    print(f"سطح اضطراب شما: {result}")
    save_test_result(user_email, "تست اضطراب", score, result)

#Step 13: Viewing Chat History

In [ ]:

def view_history(user_email):
    print("تاریخچه گفتگو:")
    get_conversation_history(user_email)

#Step 14: Main Program Loop

In [ ]:
# منوی اصلی
def main():
    init_db() # ساخت جداول پایگاه داده
    user_email = register_user()
    while True:
        print("\\nمنو:")
        print("1. گفتگو با بات")
        print("2. تست روان‌شناسی")
        print("3. نمایش تاریخچه")
        print("4. خروج")

        choice = input("انتخاب: ")
        if choice == "1":
            chat_with_bot(user_email)
        elif choice == "2":
            run_test(user_email)
        elif choice == "3":
            view_history(user_email)
        elif choice == "4":
            print("خروج از سیستم.")
            break
        else:
            print("انتخاب نامعتبر.")

#Step 15: Entry Point

In [ ]:
if __name__== "__main__":
    main()